In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config
from collections import defaultdict
from tabulate import tabulate


I0331 03:25:43.913720 140479948564288 file_utils.py:38] PyTorch version 1.4.0 available.
Loading faiss with AVX2 support.
I0331 03:25:51.058516 140479948564288 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
I0331 03:25:51.064216 140479948564288 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [2]:
torch.cuda.set_device(2)

config = Config()

task = Task(config)
task.from_pretrained()
config

cache for SNLI-spacy.data exists
*** load SNLI-spacy.data from cache cost 50.4 seconds
cache for SNLI-glove.vec exists
*** load SNLI-glove.vec from cache cost 0.058 seconds
Load model from saved/models/SNLI-esim/best.th


Basic Args:
	--task_id=SNLI
	--embed=
	--arch=esim
	--pretrain=glove
	--_model_name=
	--mode=attack
	--dir_temp=5.0
	--gnn_type=mean
	--gnn_hop=1
	--aug_data=
	--adv_iter=0
	--adv_policy=hot
	--adv_replace_num=5
	--adv_constraint=True
	--pred_ensemble=1
	--pred_transform=
	--pred_transform_args=
	--attack_method=pwws
	--attack_vectors=counter
	--attack_data_split=dev
	--attack_size=200
	--attack_gen_adv=False
	--adv_data=nogit/AGNEWS-lstm.hotflip.adv.tsv
	--alchemist=False
	--seed=2
	--cuda=3
Deduced Args:
	--model_name=SNLI-esim
	--tokenizer=spacy

In [14]:
from allennlp.data import Instance
from allennlp.data.fields import TextField
from allennlpx.allenutil import as_sentence
allenutil.as_sentence(task.train_data[0].fields['sent1'])
def as_json(instance: Instance):
    ret = {}
    for k, v in instance.items():
        if isinstance(v, TextField):
            ret[k] = as_sentence(v)
    return ret
as_json(task.train_data[0])

{'sent1': 'A person on a horse jumps over a broken down airplane .',
 'sent2': 'A person is outdoors , on a horse .'}

In [3]:
forbidden_words = DEFAULT_IGNORE_TOKENS
forbidden_words.extend([
    line.rstrip('\n') for line in open(TASK_SPECS['SST']['banned_words'])
])
# forbidden_words += stopwords.words("english")
high_words = FreqUtil.topk_frequency(task.vocab, 50, 'most', forbidden_words)
high_words

['the',
 'a',
 'and',
 'of',
 'to',
 "'s",
 'is',
 'that',
 'in',
 'it',
 'as',
 'with',
 'an',
 'film',
 'for',
 'its',
 'movie',
 'this',
 '`',
 'you',
 'be',
 'but',
 'on',
 'by',
 'more',
 'one',
 '--',
 'at',
 'than',
 'has',
 'from',
 'about',
 'his',
 'are',
 'so',
 'all',
 'or',
 'have',
 'most',
 'out',
 'story',
 'too',
 'into',
 'up',
 'who',
 'characters',
 'i',
 'comedy',
 'if',
 'just']

In [4]:
config = Config()
config.task_id = 'SST'
task = Task(config)
task.from_pretrained()
config


# values = list(task.vocab.get_index_to_token_vocabulary().values())
values = high_words
results = defaultdict(lambda: [], {})
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    for i in range(len(result[0]['probs'])):
        results[i].extend([ele['probs'][i] for ele in result])
pairs = defaultdict(lambda: [], {})
for i in range(len(results)):
    pairs[i] = sorted(zip(values, results[i]), key=lambda x: x[1], reverse=True)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 4.23 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0137 seconds
Load model from saved/models/SST-lstm/best.th


In [5]:
config._model_name = 'SST-glstm'
task = Task(config)
task.from_pretrained()
config

# values = list(task.vocab.get_index_to_token_vocabulary().values())
values = high_words
results = defaultdict(lambda: [], {})
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    for i in range(len(result[0]['probs'])):
        results[i].extend([ele['probs'][i] for ele in result])
pairs2 = defaultdict(lambda: [], {})
for i in range(len(results)):
    pairs2[i] = sorted(zip(values, results[i]), key=lambda x: x[1], reverse=True)

cache for SST-spacy.data exists
*** load SST-spacy.data from cache cost 3.22 seconds
cache for SST-glove.vec exists
*** load SST-glove.vec from cache cost 0.0091 seconds
Load model from saved/models/SST-glstm/best.th


In [32]:
from copy import deepcopy
x=[]
xs = [x] * 2
ys = [deepcopy(ele) for ele in xs]
ys[0].append(1)
print(ys)
ys = deepcopy(xs)
ys[0].append(1)
print(ys)

[[1], []]
[[1], [1]]


In [44]:
from goto import goto, label
for i in range(9):
    for j in range(9):
        for k in range(9):
            print("I am trapped, please rescue!")
            if k == 2:
                goto .breakout # breaking out from a deeply nested loop
label .breakout
print("Freedom!")

ModuleNotFoundError: No module named 'goto'

In [6]:
for i in range(len(pairs)):
    tab = []
    for ele1, score1 in pairs[i]:
        for ele2, score2 in pairs2[i]:
            if ele1 == ele2:
                tab.append((ele1, round(score1, 2),  round(score2, 2)))
    print(tabulate(tab, headers=['word', 'normal', 'adv']))

word          normal    adv
----------  --------  -----
too             0.96   0.95
or              0.82   0.98
if              0.76   0.86
from            0.71   0.44
than            0.66   0.98
have            0.6    0.03
out             0.58   0.9
its             0.57   0.42
on              0.56   0.38
just            0.56   0.91
`               0.54   0.4
at              0.53   0.44
as              0.53   0.32
but             0.53   0.35
by              0.51   0.46
about           0.48   0.31
of              0.47   0.49
movie           0.47   0.38
one             0.47   0.41
that            0.47   0.48
for             0.47   0.3
a               0.46   0.98
so              0.45   0.13
be              0.45   0.3
most            0.45   0.91
the             0.44   0.47
more            0.44   0.02
into            0.43   0.44
with            0.42   0.45
in              0.42   0.39
his             0.42   0.16
it              0.42   0.5
all             0.41   0.17
story           0.4    0.